In [1]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

def load_and_annotate_data(image_directory, annotation_directory, batch_size, target_size, num_classes=20):
    """
    Load and annotate training or validation data for semantic segmentation.

    Parameters:
    - image_directory (str): Directory path where images are stored.
    - annotation_directory (str): Directory path where segmentation masks are stored.
    - batch_size (int): Batch size for the data generator.
    - target_size (tuple): Target size for resizing images and masks.

    Returns:
    - data_generator (tf.keras.preprocessing.image.DirectoryIterator): Data generator for training or validation.
    """
    # Image data generator with augmentation (you can customize this based on your needs)
    data_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Flow from directory with the specified batch size and target size
    train_data_generator = data_generator.flow_from_directory(
        image_directory,
        class_mode=None,
        color_mode='rgb',
        batch_size=batch_size,
        target_size=target_size,
        seed=42
    )

    # Flow from directory with the specified batch size and target size
    mask_generator = data_generator.flow_from_directory(
        annotation_directory,
        class_mode=None,
        color_mode='grayscale',  # Assuming masks are grayscale
        batch_size=batch_size,
        target_size=target_size,
        seed=42
    )
    # Convert integer indices to one-hot encoded labels
    mask_generator = (to_categorical(mask, num_classes=num_classes) for mask in mask_generator)

    # Combine the image and mask generators into one
    data_generator = zip(train_data_generator, mask_generator)

    return data_generator

In [4]:
# Directories for training and validation data:
train_image_directory = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/Data/Train/Images"
train_annotation_directory = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/Data/Train/Masks"
val_image_directory = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/Data/Val/Images"
val_annotation_directory = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/Data/Val/Masks"

batch_size = 32
target_size = (224,224)  # Adjust based on your model input size

# Load and annotate training data
train_data_generator = load_and_annotate_data(train_image_directory, train_annotation_directory, batch_size, target_size, num_classes=20)

# Load and annotate validation data
val_data_generator = load_and_annotate_data(val_image_directory, val_annotation_directory, batch_size, target_size, num_classes=20)

Found 1477 images belonging to 20 classes.
Found 1477 images belonging to 20 classes.
Found 1449 images belonging to 20 classes.
Found 1449 images belonging to 20 classes.


In [5]:
import numpy as np
import matplotlib.pyplot as plt


train_data, train_label = next(train_data_generator)
val_data, val_label = next(val_data_generator)
print(train_label.shape, val_label.shape)


(32, 224, 224, 20) (32, 224, 224, 20)


In [6]:
num_classes = 20

## Load Data and Implement Model

### VGG16

In [7]:
# !wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

In [8]:
# vgg16_weights = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/models/fine_tuned_vgg16.h5"
vgg16_weights = "C:/Users/HP/Desktop/My Papers/Conferences/FCN8/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [11]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, Input, concatenate
from keras.models import Model
def vgg16(image_input):
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(image_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    p1 = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(p1)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    p2 = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(p2)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    p3 = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(p3)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    p4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(p4)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    p5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    vgg = Model(image_input, p5)
    vgg.load_weights(vgg16_weights)  # Load VGG16 weights
    
    n = 4096 
    
    # Fully-connected layers converted to convolutional layers
    fc6 = Conv2D(4096, (7, 7), activation='relu', padding='same', name='fc6')(p5)
    fc7 = Conv2D(4096, (1, 1), activation='relu', padding='same', name='fc7')(fc6)
    
    return p1, p2, p3, p4, fc7

In [17]:
from keras.layers import Conv2DTranspose, Add, Conv2D

def fcn8_decoder(p1, p2, p3, p4, fc7, num_classes=20):
    # FCN8 decoder

    # Upsample FC7 to the size of p4
    fc7_upsampled = Conv2DTranspose(512, kernel_size=(4, 4), strides=(2, 2), padding='same', name='fc7_upsample')(fc7)

    # Add skip connection from pool4
    sum_p4 = Add()([fc7_upsampled, p4])

    # Upsample the result to the size of p3
    sum_p4_upsampled = Conv2DTranspose(256, kernel_size=(4, 4), strides=(2, 2), padding='same', name='sum_p4_upsample')(sum_p4)

    # Add skip connection from pool3
    sum_p3 = Add()([sum_p4_upsampled, p3])

    # Upsample the result to the size of p2
    sum_p3_upsampled = Conv2DTranspose(128, kernel_size=(4, 4), strides=(2, 2), padding='same', name='sum_p3_upsample')(sum_p3)

    # Add skip connection from pool2
    sum_p2 = Add()([sum_p3_upsampled, p2])

    # Upsample the result to the size of p1
    sum_p2_upsampled = Conv2DTranspose(64, kernel_size=(4, 4), strides=(2, 2), padding='same', name='sum_p2_upsample')(sum_p2)

    # Add skip connection from pool1
    sum_p1 = Add()([sum_p2_upsampled, p1])

    # Final upsampling to get the output segmentation map
    output = Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(2, 2), padding='same', activation='softmax', name='output')(sum_p1)

    return output


In [18]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, UpSampling2D, Input, concatenate, Conv2DTranspose, Add

def fcn8_model(input_shape, num_classes=20):
    # Input layer
    image_input = Input(shape=input_shape)

    # VGG16 encoder
    p1, p2, p3, p4, fc7 = vgg16(image_input)

    # FCN8 decoder
    fc8_output = fcn8_decoder(p1, p2, p3, p4, fc7, num_classes=num_classes)

    # Create the final model
    model = Model(inputs=image_input, outputs=fc8_output)

    return model

# Assuming input_shape and n_classes are defined according to your data
input_shape = (224, 224, 3)  # Adjust dimensions as needed

# Build the FCN8 model
fcn8_model = fcn8_model(input_shape, num_classes=num_classes)

# Compile the model with an appropriate optimizer, loss function, and metrics
fcn8_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
fcn8_model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 224, 224, 64)         1792      ['input_4[0][0]']             
                                                                                                  
 block1_conv2 (Conv2D)       (None, 224, 224, 64)         36928     ['block1_conv1[0][0]']        
                                                                                                  
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)         0         ['block1_conv2[0][0]']        
                                                                                            

In [19]:
# Assuming you have already defined and compiled your FCN8 model (fcn8_model)
import math
# Number of training and validation steps per epoch
train_steps_per_epoch = math.ceil(1477/batch_size)
val_steps_per_epoch = math.ceil(1449/batch_size)

# Number of epochs
epochs = 1  # You can adjust this based on your needs

# Fit the model using fit_generator
history = fcn8_model.fit(
    train_data_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    validation_data=val_data_generator,
    validation_steps=val_steps_per_epoch
)

# Optionally, you can save the trained model
fcn8_model.save('fcn8_model.h5')


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


KeyboardInterrupt: 